#**OmegaFold**
(DDA: this notebook was copied from the notebook on colabfold version d991ad9b2d14fbcaf660a6955a9a9de34b4f5eb8).

for more details see: [Github](https://github.com/HeliXonProtein/OmegaFold), [Preprint](https://www.biorxiv.org/content/10.1101/2022.07.21.500999v1)

#### **Tips and Instructions**
- click the little ▶ play icon to the left of each cell below.
- use "/" to specify chainbreaks, (eg. sequence="AAA/AAA")
- for homo-oligomeric predictions, set copies > 1


In [1]:
#@markdown ##Install
import os,sys,re
from IPython.utils import io
if "SETUP_DONE" not in dir():
  import torch
  device = "cuda" if torch.cuda.is_available() else "cpu"
  with io.capture_output() as captured:
    if not os.path.isdir("OmegaFold"):
      %shell git clone --branch beta --quiet https://github.com/sokrypton/OmegaFold.git
      # %shell cd OmegaFold; pip -q install -r requirements.txt
      %shell pip -q install py3Dmol biopython
      %shell apt-get install aria2 -qq > /dev/null
      %shell aria2c -q -x 16 https://helixon.s3.amazonaws.com/release1.pt
      %shell mkdir -p ~/.cache/omegafold_ckpt
      %shell mv release1.pt ~/.cache/omegafold_ckpt/model.pt
  SETUP_DONE = True

In [5]:
#@markdown ##Run **OmegaFold**
from string import ascii_uppercase, ascii_lowercase
import hashlib

def get_hash(x): return hashlib.sha1(x.encode()).hexdigest()
alphabet_list = list(ascii_uppercase+ascii_lowercase)

jobname = "test" #@param {type:"string"}
jobname = re.sub(r'\W+', '', jobname)[:50]

sequence = "PIAQIHILEGRSDEQKETLIREVSEAISRSLDAPLTSVRVIITEMAKGHFGIGGELASK" #@param {type:"string"}
sequence = re.sub("[^A-Z:]", "", sequence.replace("/",":").upper())
sequence = re.sub(":+",":",sequence)
sequence = re.sub("^[:]+","",sequence)
sequence = re.sub("[:]+$","",sequence)

copies = 1 #@param {type:"integer"}
sequence = ":".join([sequence] * copies)
#@markdown **Advanced Options**
num_cycle = 4 #@param ["1", "2", "4", "8", "16", "32"] {type:"raw"}
offset_rope = False #@param {type:"boolean"}

ID = jobname+"_"+get_hash(sequence)[:5]
seqs = sequence.split(":")
lengths = [len(s) for s in seqs]

def get_subbatch_size(L):
  if L <  500: return 500
  if L < 1000: return 200
  return 150
subbatch_size = get_subbatch_size(sum(lengths))

u_seqs = list(set(seqs))

if len(seqs) == 1: mode = "mono"
elif len(u_seqs) == 1: mode = "homo"
else: mode = "hetero"

with open(f"{ID}.fasta","w") as out:
  out.write(f">{ID}\n{sequence}\n")

%shell python OmegaFold/main.py --offset_rope={offset_rope} --device={device} --subbatch_size={subbatch_size} AAT.fasta .

def renum_pdb_str(pdb_str, Ls=None, renum=True, offset=1):
  if Ls is not None:
    L_init = 0
    new_chain = {}
    for L,c in zip(Ls, alphabet_list):
      new_chain.update({i:c for i in range(L_init,L_init+L)})
      L_init += L  

  n,num,pdb_out = 0,offset,[]
  resnum_ = None
  chain_ = None
  new_chain_ = new_chain[0]
  for line in pdb_str.split("\n"):
    if line[:4] == "ATOM":
      chain = line[21:22]
      resnum = int(line[22:22+5])
      if resnum_ is None: resnum_ = resnum
      if chain_ is None: chain_ = chain
      if resnum != resnum_ or chain != chain_:
        num += (resnum - resnum_)  
        n += 1
        resnum_,chain_ = resnum,chain
      if Ls is not None:
        if new_chain[n] != new_chain_:
          num = offset
          new_chain_ = new_chain[n]
      N = num if renum else resnum
      if Ls is None: pdb_out.append("%s%4i%s" % (line[:22],N,line[26:]))
      else: pdb_out.append("%s%s%4i%s" % (line[:21],new_chain[n],N,line[26:]))        
  return "\n".join(pdb_out)

pdb_str = renum_pdb_str(open(f"{ID}.pdb",'r').read(), Ls=lengths)
with open(f"{ID}.pdb","w") as out:
  out.write(pdb_str)

INFO:root:Loading weights from /root/.cache/omegafold_ckpt/model.pt
INFO:root:Constructing OmegaFold
INFO:root:Reading AAT.fasta
INFO:root:Predicting 1th chain in AAT.fasta
INFO:root:396 residues in this chain.
INFO:root:Finished prediction in 334.29 seconds.
INFO:root:Saving prediction to ./WT.pdb
INFO:root:Saved
INFO:root:Predicting 2th chain in AAT.fasta
INFO:root:396 residues in this chain.
INFO:root:Finished prediction in 334.64 seconds.
INFO:root:Saving prediction to ./HEX.pdb
INFO:root:Saved
INFO:root:Predicting 3th chain in AAT.fasta
INFO:root:396 residues in this chain.
INFO:root:Finished prediction in 334.73 seconds.
INFO:root:Saving prediction to ./IYIT.pdb
INFO:root:Saved
INFO:root:Predicting 4th chain in AAT.fasta
INFO:root:396 residues in this chain.
INFO:root:Finished prediction in 334.84 seconds.
INFO:root:Saving prediction to ./VFIT.pdb
INFO:root:Saved
INFO:root:Predicting 5th chain in AAT.fasta
INFO:root:396 residues in this chain.
INFO:root:Finished prediction in 334